# Gaining familiarity with web3py

In this notebook I will work through the examples provided in the [web3py documentation](https://web3py.readthedocs.io/en/stable/examples.html) to get a better feeling for the functionality.  

Ultimately my goal is to be able to compile and deploy Solidity code directly from within Python. Installing the solc solidity commandline compiler via homebrew is a huge step towards making this happen.

### Steps taken

* Installed Anaconda (python 3.6)
* virtualenv with conda
    * install web3 into venv via pip
* [install solidity compiler](https://solidity.readthedocs.io/en/latest/installing-solidity.html#binary-packages) via homebrew
    * ```brew tap ethereum/ethereum```
    * ```brew install solidity```
    * Install Solidity 0.4.25 (no commit available for 0.4.26)
        * ```brew install https://raw.githubusercontent.com/ethereum/homebrew-ethereum/f26f126820e5f47c3ed7ec6d5e6e046707443d87/solidity.rb```
        * discovered via instructions [here](https://solidity.readthedocs.io/en/v0.4.21/installing-solidity.html)
* install NPM 
* install ganache_cli via npm
    * ```npm install -g ganache-cli```

### Preparation: Connect Python to local Ganache Node

We start an instance of Ganache, either via the GUI or the command line interface:  
```ganache-cli -p 7545```

In [6]:
import time
from web3 import Web3, HTTPProvider

# Ganache Connection
w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:8545"))

# Wallet address
wallet_address       = "0x048a4fabe0d6a1aec7fc818867f53d1022a55d81"
wallet_address       = w3.toChecksumAddress(wallet_address)

# Private key
wallet_private_key   = "0xd234b1cc3a170f921647d96d4a91c790158fff0c0ebd5c92f49ed58e70a4ae57"

# Contract address from Ganache
contract_address     = "0x9B3da536bfFf54974AE3D9151D7C6F5dBE81990E"


### Send ether

In [3]:
def send_ether(amount_in_ether, recipient_address):
    amount_in_wei = w3.toWei(amount_in_ether,'ether');

    # How many transactions have been made by wallet?
    nonce = w3.eth.getTransactionCount(wallet_address)
    
    # Specify transcation details
    txn_dict = {
            'to': recipient_address,
            'value': amount_in_wei,
            'gas': 2000000,
            'gasPrice': w3.toWei('40', 'gwei'),
            'nonce': nonce,
            'chainId': 3
    }
    
    # Sign transaction
    signed_txn = w3.eth.account.signTransaction(txn_dict, wallet_private_key)

    # Send transaction & store transaction hash
    txn_hash = w3.eth.sendRawTransaction(signed_txn.rawTransaction)

    # Check if transaction was added to blockchain
    time.sleep(0.5)
    txn_receipt = w3.eth.getTransactionReceipt(txn_hash)
    return txn_hash

In [4]:
recipient = "0xa6aea4b3aba0181f741c0943d30db5f869c72f1f"
recipient = w3.toChecksumAddress(recipient)

In [7]:
txn_hash = send_ether(1,recipient)

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=8545): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f94b5c15240>: Failed to establish a new connection: [Errno 111] Connection refused'))

### Look up a block

In [31]:
w3.eth.getBlock(1)

AttributeDict({'number': 1,
 'hash': HexBytes('0x971b2785b4f790f78be3b70082a08b892d88962460ad4577d9be8698f3dbd096'),
 'parentHash': HexBytes('0xe24bf39bc63897eb15a96a6cb5159dd3252008468e8957bb505937417767712a'),
 'mixHash': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'),
 'nonce': HexBytes('0x0000000000000000'),
 'sha3Uncles': HexBytes('0x1dcc4de8dec75d7aab85b567b6ccd41ad312451b948a7413f0a142fd40d49347'),
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'transactionsRoo

In [32]:
# Retrieve the last block
w3.eth.getBlock('latest')

AttributeDict({'number': 2,
 'hash': HexBytes('0x97bacf6320ff705ea9a9a6ea635c76722cb998d5c19f49a32cee5b545432a721'),
 'parentHash': HexBytes('0x971b2785b4f790f78be3b70082a08b892d88962460ad4577d9be8698f3dbd096'),
 'mixHash': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'),
 'nonce': HexBytes('0x0000000000000000'),
 'sha3Uncles': HexBytes('0x1dcc4de8dec75d7aab85b567b6ccd41ad312451b948a7413f0a142fd40d49347'),
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'transactionsRoo

In [34]:
# directly obtain the current block number
w3.eth.blockNumber

2

### Loop up a transaction via hash

In [39]:
w3.eth.getTransaction(txn_hash)

AttributeDict({'hash': HexBytes('0xc8e96c40a99847465f95bf7e243bdb7e2e6d807aa3d3e9890da1afc0709c5ad6'),
 'nonce': 3,
 'blockHash': HexBytes('0xd8cbbab0a99d469e367302315c843af91ff7a775f6945474b27356f10ec215e8'),
 'blockNumber': 4,
 'transactionIndex': 0,
 'from': '0x04987eC887acc216E9182FdD9342F91F520F38c3',
 'to': '0x3bA6c1ceC0459CF72254a03552B0934D88b63BB8',
 'value': 1000000000000000000,
 'gas': 2000000,
 'gasPrice': 40000000000,
 'input': '0x',
 'v': 41,
 'r': HexBytes('0x751e84bb37259fb58d955e87116a8bb9be04d1e1aee322c6bdf5579d796b3b61'),
 's': HexBytes('0x307b9939231725e54d122d5f69b07f5f2372d997259a483a3b7bea9239d54ddb')})

---

### Deploy a contract

Important that py-solc is installed for code to execute :) Still an error with the execution of the seoncd cell but I can look at that tomorrow

In [44]:
!{sys.executable} -m pip install py-solc

  Using cached https://files.pythonhosted.org/packages/47/74/d36abca3f36ccdcd04976c50f83502c870623e5beb4a4ec96c7bad4bb9e8/py_solc-3.2.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/28/be/3a7241d731ba89063780279a5433f5971c1cf41735b64a9f874b7c3ff995/semantic_version-2.6.0-py3-none-any.whl


In [77]:
!{sys.executable} -m pip install py-solc-x

In [ ]:
import sys
import time
import pprint

from web3.providers.eth_tester import EthereumTesterProvider
from web3 import Web3
from solcx import compile_source


def compile_source_file(file_path):
    with open(file_path, 'r') as f:
        source = f.read()

    return compile_source(source)

In [90]:
contract_source_path = 'contract.sol'
compiled_sol = compile_source_file('contract.sol')

In [92]:
contract_id, contract_interface = compiled_sol.popitem()

In [93]:
contract_id

'<stdin>:Mortal'

In [73]:
# Interface contains ABI and more
contract_interface

{'abi': [{'constant': False,
   'inputs': [{'name': '_greeting', 'type': 'string'}],
   'name': 'setGreeting',
   'outputs': [],
   'payable': False,
   'stateMutability': 'nonpayable',
   'type': 'function'},
  {'constant': True,
   'inputs': [],
   'name': 'greet',
   'outputs': [{'name': '', 'type': 'string'}],
   'payable': False,
   'stateMutability': 'view',
   'type': 'function'},
  {'constant': True,
   'inputs': [],
   'name': 'greeting',
   'outputs': [{'name': '', 'type': 'string'}],
   'payable': False,
   'stateMutability': 'view',
   'type': 'function'},
  {'inputs': [],
   'payable': False,
   'stateMutability': 'nonpayable',
   'type': 'constructor'}],
 'asm': {'.code': [{'begin': 25, 'end': 322, 'name': 'PUSH', 'value': '80'},
   {'begin': 25, 'end': 322, 'name': 'PUSH', 'value': '40'},
   {'begin': 25, 'end': 322, 'name': 'MSTORE'},
   {'begin': 77, 'end': 133, 'name': 'CALLVALUE'},
   {'begin': 8, 'end': 17, 'name': 'DUP1'},
   {'begin': 5, 'end': 7, 'name': 'ISZERO'

Up to this point everything seems to work. We can compile the contract and also have the abi and binary code conveniently stored in our ```contract_interface```.

---

In [97]:
abi=contract_interface['abi']
bytecode=contract_interface['bin']

In [102]:
w3.eth.contract(bytecode).deploy()

BadFunctionCallOutput: Could not transact with/call contract function, is contract deployed correctly and chain synced?

In [79]:
def deploy_contract(w3, contract_interface):
    tx_hash = w3.eth.contract(
        abi=contract_interface['abi'],
        bytecode=contract_interface['bin']).deploy()

    address = w3.eth.getTransactionReceipt(tx_hash)['contractAddress']
    return address


def wait_for_receipt(w3, tx_hash, poll_interval):
    while True:
        tx_receipt = w3.eth.getTransactionReceipt(tx_hash)
        if tx_receipt:
            return tx_receipt
        time.sleep(poll_interval)

In [94]:
contract_interface['abi']

[{'constant': False,
  'inputs': [],
  'name': 'kill',
  'outputs': [],
  'payable': False,
  'stateMutability': 'nonpayable',
  'type': 'function'},
 {'inputs': [],
  'payable': False,
  'stateMutability': 'nonpayable',
  'type': 'constructor'}]

In [95]:
address = deploy_contract(w3, contract_interface)

/anaconda3/envs/conda_luce/lib/python3.7/site-packages/ipykernel_launcher.py:20: DeprecationWarning: deploy is deprecated in favor of contract.constructor.transact


ValueError: {'message': 'from not found; is required', 'code': -32000, 'data': {'stack': 'n: from not found; is required\n    at w.queueTransaction (/usr/local/lib/node_modules/ganache-cli/build/ganache-core.node.cli.js:52:200847)\n    at u.eth_sendTransaction (/usr/local/lib/node_modules/ganache-cli/build/ganache-core.node.cli.js:52:191109)\n    at u.handleRequest (/usr/local/lib/node_modules/ganache-cli/build/ganache-core.node.cli.js:52:188272)\n    at t (/usr/local/lib/node_modules/ganache-cli/build/ganache-core.node.cli.js:52:51836)\n    at a.handleRequest (/usr/local/lib/node_modules/ganache-cli/build/ganache-core.node.cli.js:52:186636)\n    at t (/usr/local/lib/node_modules/ganache-cli/build/ganache-core.node.cli.js:52:51836)\n    at f.s.handleRequest (/usr/local/lib/node_modules/ganache-cli/build/ganache-core.node.cli.js:52:179766)\n    at f.handleRequest (/usr/local/lib/node_modules/ganache-cli/build/ganache-core.node.cli.js:52:177592)\n    at t (/usr/local/lib/node_modules/gan

In [64]:
print("Deployed {0} to: {1}\n".format(contract_id, address))

/anaconda3/envs/conda_luce/lib/python3.7/site-packages/ipykernel_launcher.py:20: DeprecationWarning: deploy is deprecated in favor of contract.constructor.transact


ValueError: {'message': 'from not found; is required', 'code': -32000, 'data': {'stack': 'n: from not found; is required\n    at w.queueTransaction (/usr/local/lib/node_modules/ganache-cli/build/ganache-core.node.cli.js:52:200847)\n    at u.eth_sendTransaction (/usr/local/lib/node_modules/ganache-cli/build/ganache-core.node.cli.js:52:191109)\n    at u.handleRequest (/usr/local/lib/node_modules/ganache-cli/build/ganache-core.node.cli.js:52:188272)\n    at t (/usr/local/lib/node_modules/ganache-cli/build/ganache-core.node.cli.js:52:51836)\n    at a.handleRequest (/usr/local/lib/node_modules/ganache-cli/build/ganache-core.node.cli.js:52:186636)\n    at t (/usr/local/lib/node_modules/ganache-cli/build/ganache-core.node.cli.js:52:51836)\n    at f.s.handleRequest (/usr/local/lib/node_modules/ganache-cli/build/ganache-core.node.cli.js:52:179766)\n    at f.handleRequest (/usr/local/lib/node_modules/ganache-cli/build/ganache-core.node.cli.js:52:177592)\n    at t (/usr/local/lib/node_modules/gan

In [63]:
store_var_contract = w3.eth.contract(
   address=address,
   abi=contract_interface['abi'])

gas_estimate = store_var_contract.functions.setVar(255).estimateGas()
print("Gas estimate to transact with setVar: {0}\n".format(gas_estimate))

if gas_estimate < 100000:
    print("Sending transaction to setVar(255)\n")
    tx_hash = store_var_contract.functions.setVar(255).transact()
    receipt = wait_for_receipt(w3, tx_hash, 1)
    print("Transaction receipt mined: \n")
    pprint.pprint(dict(receipt))
else:
    print("Gas cost exceeds 100000")

/anaconda3/envs/conda_luce/lib/python3.7/site-packages/ipykernel_launcher.py:20: DeprecationWarning: deploy is deprecated in favor of contract.constructor.transact


ValueError: {'message': 'from not found; is required', 'code': -32000, 'data': {'stack': 'n: from not found; is required\n    at w.queueTransaction (/usr/local/lib/node_modules/ganache-cli/build/ganache-core.node.cli.js:52:200847)\n    at u.eth_sendTransaction (/usr/local/lib/node_modules/ganache-cli/build/ganache-core.node.cli.js:52:191109)\n    at u.handleRequest (/usr/local/lib/node_modules/ganache-cli/build/ganache-core.node.cli.js:52:188272)\n    at t (/usr/local/lib/node_modules/ganache-cli/build/ganache-core.node.cli.js:52:51836)\n    at a.handleRequest (/usr/local/lib/node_modules/ganache-cli/build/ganache-core.node.cli.js:52:186636)\n    at t (/usr/local/lib/node_modules/ganache-cli/build/ganache-core.node.cli.js:52:51836)\n    at f.s.handleRequest (/usr/local/lib/node_modules/ganache-cli/build/ganache-core.node.cli.js:52:179766)\n    at f.handleRequest (/usr/local/lib/node_modules/ganache-cli/build/ganache-core.node.cli.js:52:177592)\n    at t (/usr/local/lib/node_modules/gan